# Dev2

In [1]:
using Pkg;
Pkg.activate("../../SchrodingerEquationSolver");
include("../src/SchrodingerEquationSolver.jl");
import .SchrodingerEquationSolver as ses
import .ses.Grids as Grids
import .ses.Potentials as Potentials
import .ses.MathUtils as MathUtils
import .ses.Hydrogen as Hydrogen
import .ses.InitialConditions as InitialConditions
import .ses.OneDSchrodingerEquationSolver as odses
import .ses.OneDPoissonEquationSolver as odpes
import .ses.EigenvalueFinders as EigenvalueFinders
import .ses.AtomBasisSet as AtomBasisSet
import .ses.Density as Density
import .ses.ExchangeCorrelation as ExchangeCorrelation
using Plots

  Activating project at `~/Desktop/physics_coding_projects/julia/SchrodingerEquationSolver`


In [2]:
#Define parameters and produce an exponential grid.
r_max::Float64=50.0;#Max radius of space grid.
Z::Int64=3;#Atomic number, also used as the charge of coulomb potential.
#b::Float64=0.001504; #By decreasing b the number of points in the grid increase.
grid= ses.Grids.exponential_grid(r_max, Z);
grid_sqrt= grid.^2.0; 
N= size(grid)[1]; #Number of points in the grid.

In [3]:
#Initialization of potentials and energies

#Initializing coulomb potential due to nuclei charge.
V_colu::Vector{Float64}= Potentials.coulomb_potential(Z, grid);
#Initializing Hartree potential due to electron density. 
V_hartree::Vector{Float64}=zeros(Float64, N);
#Initializing exchange potential.
V_x::Vector{Float64}=zeros(Float64, N);
#Initializing correlation potential.
V_c::Vector{Float64}=zeros(Float64, N);
#Initializing energy exchange potential.
E_xp::Vector{Float64}=zeros(Float64, N);
#Initializing energy correlation potential.
E_cp::Vector{Float64}=zeros(Float64, N);
#Initializing exchange + correlation potential.
V_xcp::Vector{Float64}=zeros(Float64, N);
#Initializing the density.
density_in::Vector{Float64}= zeros(Float64, N);
#Initializing total energy
E_total::Float64=1.0;
#Initializing total energy step before
E_total_before::Float64=2.0;
#Initializing energy from energy eigenvalues
E_eigen::Float64=0.0;
#Initializing exchange correlation potential value after integral
V_xc::Float64= 0.0;
#Initializing Hartree energy
E_hartree::Float64= 0.0;
#Initializing exchange energy
E_x::Float64= 0.0;
#Initializing correlation enrgy
E_c::Float64= 0.0;
#Initializing basis set data structure
basis= AtomBasisSet.init_atom_basis_set(Z, grid);

In [4]:
#Energy minimization loop 
while abs(E_total - E_total_before) > 10.0e-12
    E_eigen=0.0;
    #Loop over every orbital to solve independent particle Schrodinger equation.
    for i_orbi in basis.orbitals
        #angular potential for l orbital
        V_angu= Potentials.angular_potential(i_orbi.l, grid);
        #Assemble effective potential.
        V_effe= V_colu .+ V_angu .+ V_hartree .+ V_x .+ V_c;
        #Energy grid to search for new eigenvalue, search around the previous eigenvalue.
        E_grid= Grids.uniform_grid(i_orbi.E - 0.5*i_orbi.E, i_orbi.E + 0.5*i_orbi.E, 250);
        #Find the energy intervals with potential eigenvalues.
        E_intervals, bad_intervals= EigenvalueFinders.find_eigenvalue_intervals(E_grid, V_effe,
                                                        grid,InitialConditions.atom, i_orbi.l);
        #Search for a solution to the independent particle Schrodinger equation for the first energy interval, 
        #there should be only one interval. Any other interval would be of a higher energy.
        u_temp, ei_temp= EigenvalueFinders.illinois_eigenvalue_finder(E_intervals[1], V_effe, grid,InitialConditions.atom);
        #Update eigenvalue and eigenfunction in the basis set data structure.
        i_orbi.E=ei_temp;
        i_orbi.u=u_temp;
        E_eigen+= i_orbi.occu*ei_temp;

    end
    #Update E_total_before from the E_total from the previous step.
    E_total_before= float(E_total);
    #Calculate density with new basis set.
    density_out= Density.calculate_density(basis);
    #Smooth the density with linear mixing (combination) of the previous and current densities.
    density_in= Density.linear_mixing(density_in, density_out, alpha=0.25);
    #Solve Poisson equation to find the new Hartree potential.
    V_hartree= odpes.solver(Z, density_in, grid);
    #Calculate new exchange and correlation potentials.
    V_x, E_xp, V_c, E_cp= ExchangeCorrelation.potentials(density_in);
    #Add exchange and correlation potentials.
    V_xcp= V_x .+ V_c;
    #Integrals to calculate energy components.
    V_xc= (4.0*pi)*MathUtils.integral((V_xcp.*density_in.*grid_sqrt), grid);
    E_hartree= 0.5*(4.0*pi)*(MathUtils.integral((V_hartree.*density_in.*grid_sqrt), grid));
    E_x= (4.0*pi)*MathUtils.integral((E_xp.*density_in.*grid_sqrt), grid);
    E_c= (4.0*pi)*MathUtils.integral((E_cp.*density_in.*grid_sqrt), grid);
    #C_in= (4.0*pi)*MathUtils.integral((density_in.*grid_sqrt), grid)
    #C_out= (4.0*pi)*MathUtils.integral((density_out.*grid_sqrt), grid)
    #Calculate total energy.
    E_total= E_eigen - E_hartree + E_x + E_c - V_xc;
end

In [5]:
E_total

-7.335200590961793

In [6]:
save_path="../save_basis_set/test_write_basis.json"
AtomBasisSet.save_basis_set(basis, save_path);

In [13]:
for e in basis.orbitals
    fieldnames(e)
end

MethodError: MethodError: no method matching fieldnames(::Main.SchrodingerEquationSolver.AtomBasisSet.orbital)
Closest candidates are:
  fieldnames(!Matched::Core.TypeofBottom) at reflection.jl:188
  fieldnames(!Matched::Type{<:Tuple}) at reflection.jl:190
  fieldnames(!Matched::DataType) at reflection.jl:185
  ...

In [9]:
fieldnames(basis)

MethodError: MethodError: no method matching fieldnames(::Main.SchrodingerEquationSolver.AtomBasisSet.atom_basis_set)
Closest candidates are:
  fieldnames(!Matched::Core.TypeofBottom) at reflection.jl:188
  fieldnames(!Matched::Type{<:Tuple}) at reflection.jl:190
  fieldnames(!Matched::DataType) at reflection.jl:185
  ...

In [20]:
Ref(basis)

Base.RefValue{Main.SchrodingerEquationSolver.AtomBasisSet.atom_basis_set}(Main.SchrodingerEquationSolver.AtomBasisSet.atom_basis_set([3.336962704870589e-9, 6.681622635612358e-9, 1.0033997547079788e-8, 1.3394105235080599e-8, 1.676196353647307e-8, 2.0137590329257294e-8, 2.3521003532672462e-8, 2.691222110729168e-8, 3.031126105511636e-8, 3.371814141967263e-8  …  48.97703290195987, 49.0900060842293, 49.20323985679543, 49.31673482075066, 49.430491578573395, 49.54451073413226, 49.65879289268884, 49.773338660900244, 49.88814864682354, 50.00322345991833], Main.SchrodingerEquationSolver.AtomBasisSet.orbital[Main.SchrodingerEquationSolver.AtomBasisSet.orbital(1, 0, "1s", -1.8805981493310349, 2.0, [3.055574565500322e-8, 6.11819725786213e-8, 9.187884302880042e-8, 1.2264651997441103e-7, 1.5348516674485708e-7, 1.8439494704401665e-7, 2.153760249539114e-7, 2.4642856492907126e-7, 2.7755273180568535e-7, 3.087486907993262e-7  …  5.245362156811454e-41, 4.2110530630466335e-41, 3.3780022893868104e-41, 2.7071

In [19]:
fieldnames(AtomBasisSet.atom_basis_set)

(:grid, :orbitals)

In [29]:
temp1= Dict(fieldnames(AtomBasisSet.orbital) .=> getfield.(Ref(basis.orbitals[1]), fieldnames(AtomBasisSet.orbital)))

Dict{Symbol, Any} with 6 entries:
  :l    => 0
  :n    => 1
  :occu => 2.0
  :name => "1s"
  :u    => [3.05557e-8, 6.1182e-8, 9.18788e-8, 1.22647e-7, 1.53485e-7, 1.84395e…
  :E    => -1.8806

In [23]:
using JSON3

In [30]:
JSON3.write(temp1)

"{\"l\":0,\"n\":1,\"occu\":2.0,\"name\":\"1s\",\"u\":[3.055574565500322e-8,6.11819725786213e-8,9.187884302880042e-8,1.2264651997441103e-7,1.5348516674485708e-7,1.8439494704401665e-7,2.153760249539114e-7,2.4642856492907126e-7,2.7755273180568535e-7,3.087486907993262e-7,3.400166075067" ⋯ 155836 bytes ⋯ "529201724042769e-41,5.245362156811454e-41,4.2110530630466335e-41,3.3780022893868104e-41,2.7071459588803783e-41,2.166874107740604e-41,1.731610845545307e-41,1.3806641708308273e-41,1.0972303598732395e-41,8.676910097098285e-42,6.80958857437093e-42],\"E\":-1.8805981493310349}"

In [31]:
basis.grid

7534-element Vector{Float64}:
  3.336962704870589e-9
  6.681622635612358e-9
  1.0033997547079788e-8
  1.3394105235080599e-8
  1.676196353647307e-8
  2.0137590329257294e-8
  2.3521003532672462e-8
  2.691222110729168e-8
  3.031126105511636e-8
  3.371814141967263e-8
  ⋮
 49.0900060842293
 49.20323985679543
 49.31673482075066
 49.430491578573395
 49.54451073413226
 49.65879289268884
 49.773338660900244
 49.88814864682354
 50.00322345991833

In [33]:
out= Dict{String, String}("grid"=>JSON3.write(basis.grid))

Dict{String, String} with 1 entry:
  "grid" => "[3.336962704870589e-9,6.681622635612358e-9,1.0033997547079788e-8,1…